In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

≈
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import string 
string.punctuation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords.words('english')
import re
import unicodedata
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from nltk.tokenize import word_tokenize
from sklearn import linear_model
from sklearn import metrics
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
df_train=pd.read_csv('../input/nlp-getting-started/train.csv')
df_train.head()

In [ ]:
df_test=pd.read_csv('../input/nlp-getting-started/test.csv')
df_test.head()

In [ ]:
df_sample=pd.read_csv('../input/nlp-getting-started/sample_submission.csv')
df_sample.head()

## Date preprocessing

In [ ]:
df_train.info()

In [ ]:
df_train['target'].value_counts()

In [ ]:
df_train['target'].value_counts().plot(kind='bar')

In [ ]:
df_train.head()

In [ ]:
df_train['keyword'].value_counts()

In [ ]:
df_train.groupby(['keyword']).mean()

In [ ]:
df_train.describe(include='all')

In [ ]:
df_train.describe()

### Clean the text

In [ ]:
EMOJIS = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}
URLPATTERN        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
USERPATTERN       = '@[^\s]+'
SEQPATTERN   = r"(.)\1\1+"
SEQREPLACE = r"\1\1"

In [ ]:
def tweets_cleaning(text):
    lowercase = text.lower()
    punc_removal = [char for char in lowercase if char not in string.punctuation]
    punc_removal_joined = ''.join(punc_removal)
    url_removal = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', punc_removal_joined, flags=re.MULTILINE)
    for emoji in EMOJIS.keys():
        url_removal = url_removal.replace(emoji, "EMOJI" + EMOJIS[emoji])  
   # emoji_removal = url_removal.encode('ascii', 'ignore').decode('ascii')
    emoji_removal=url_removal
    stopwords_removal = [word for word in emoji_removal.split() if word not in stopwords.words('english')]
    return stopwords_removal

In [ ]:
df_train['cleaned_text']=df_train['text'].apply(tweets_cleaning).astype(str)

In [ ]:
df_test['cleaned_text']=df_test['text'].apply(tweets_cleaning).astype(str)

In [ ]:
df_train.head()

In [ ]:
df_train_clean=df_train[['cleaned_text','target']]

In [ ]:
df_train_clean.head()

### Tokenization

In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df_train_clean['cleaned_text'].values)
X = tokenizer.texts_to_sequences(df_train_clean['cleaned_text'].values)
X = pad_sequences(X)

## LSTM Model training

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
Y = pd.get_dummies(df_train_clean['target']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

In [ ]:
batch_size = 16
model.fit(X_train, Y_train, epochs = 19, batch_size=batch_size, verbose = 2)

## Predictions

In [ ]:
df_test.head()

In [ ]:
df_test.info()

In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df_test['cleaned_text'].values)
X = tokenizer.texts_to_sequences(df_test['cleaned_text'].values)
X = pad_sequences(X)

In [ ]:
pred=model.predict(X)

In [ ]:
labels=[0,1]

In [ ]:
pred.shape

In [ ]:
pred

In [ ]:
tweet_predict=[]

In [ ]:
df_test.head()

In [ ]:
df_test['id'][0]

In [ ]:
idx=0
for p in pred:
    predict=[]
    t_id=df_test['id'][idx]
    idx+=1
    target=labels[np.argmax(p)]
    predict.append(t_id)
    predict.append(target)
    tweet_predict.append(predict)
    

In [ ]:
df_sub=pd.DataFrame(tweet_predict,columns=["id","target"])

In [ ]:
df_sub.head()

In [ ]:
df_sub['target'].value_counts()

In [ ]:
df_sub.to_csv('submission_lstm.csv',index=False)

### LSTM Ends

## Save prediction

In [ ]:
def save_prediction(pred,model):
    df_test=pd.read_csv('../input/nlp-getting-started/test.csv')
    submission_list=[]
    idx=0
    tweet_predict=[]
    for p in pred:
        predict=[]
        t_id=df_test['id'][idx]
        idx+=1
        target=p
        predict.append(t_id)
        predict.append(target)
        tweet_predict.append(predict)
    df_sub=pd.DataFrame(tweet_predict,columns=['id','target'])
    df_sub.to_csv('submission_'+model+".csv",index=False)
    print(model+"- model prediction")
    print(df_sub['target'].value_counts())
    
        

# Logistic regression +CounteVcetorizer

##### We create a new column called kfold and fill it with -1

In [ ]:
df_train["kfold"]=-1

#### randomize the row of the data

In [ ]:
df_train=df_train.sample(frac=1).reset_index(drop=True)

#### Fetch labels

In [ ]:
y=df_train.target

#### initiate the kfold class from model_selection module

In [ ]:
kf=model_selection.StratifiedKFold(n_splits=4)

In [ ]:
for f,(t_,v_) in enumerate(kf.split(X=df_train,y=y)):
    df_train.loc[v_,'kfold']=f

In [ ]:
df_train.head()

In [ ]:
df_train['kfold'].value_counts()

In [ ]:
for fold_ in range(5):
    train_df=df_train[df_train.kfold!=fold_].reset_index(drop=True)
    if fold_==4:
        train_df=df_train.reset_index(drop=True)
        test_df=df_test.reset_index(drop=True)
    else:
        test_df=df_train[df_train.kfold==fold_].reset_index(drop=True)
    ##### initilize CountVectorizer with NLTK's word_tokenize
    ##### function as tokenizer
    count_vec=CountVectorizer(
        tokenizer=word_tokenize,
        token_pattern=None)
    #### fit count_vec on training data reviews
    count_vec.fit(train_df.text)
    #transform training and validation data reviews
    x_train=count_vec.transform(train_df.text)
    x_test=count_vec.transform(test_df.text)
    # initialize logistic regression model
    model_logistic=linear_model.LogisticRegression(max_iter=1000)
    # fit the model on training data reviews and sentiment
    model_logistic.fit(x_train,train_df.target)
    preds_log=model_logistic.predict(x_test)
    ##Calculate accuracy
    if fold_!=4:
        accuracy=metrics.accuracy_score(test_df.target,preds_log)
        print(f"Fold:{fold_}")
        print(f"Accuracy={accuracy}")
        print("")
    

### Prediction

In [ ]:
set(preds_log)

In [ ]:
len(preds_log)

In [ ]:
save_prediction(preds_log,model='logistic')

# Naive bayes

In [ ]:
from sklearn import naive_bayes

In [ ]:
for fold_ in range(5):
    train_df=df_train[df_train.kfold!=fold_].reset_index(drop=True)
    if fold_==4:
        train_df=df_train.reset_index(drop=True)
        test_df=df_test.reset_index(drop=True)
    else:
        test_df=df_train[df_train.kfold==fold_].reset_index(drop=True)
    ##### initilize CountVectorizer with NLTK's word_tokenize
    ##### function as tokenizer
    count_vec=CountVectorizer(
        tokenizer=word_tokenize,
        token_pattern=None)
    #### fit count_vec on training data reviews
    count_vec.fit(train_df.text)
    #transform training and validation data reviews
    x_train=count_vec.transform(train_df.text)
    x_test=count_vec.transform(test_df.text)
    # initialize naive bayes model
    model_naive=naive_bayes.MultinomialNB()
    # fit the model on training data reviews and sentiment
    model_naive.fit(x_train,train_df.target)
    preds_naive=model_naive.predict(x_test)
    ##Calculate accuracy
    if fold_!=4:
        accuracy=metrics.accuracy_score(test_df.target,preds_naive)
        print(f"Fold:{fold_}")
        print(f"Accuracy={accuracy}")
        print("")
    

### Prediction

In [ ]:
save_prediction(preds_naive,model='naive')

# Logistic +TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
for fold_ in range(5):
    train_df=df_train[df_train.kfold!=fold_].reset_index(drop=True)
    if fold_==4:
        train_df=df_train.reset_index(drop=True)
        test_df=df_test.reset_index(drop=True)
    else:
        test_df=df_train[df_train.kfold==fold_].reset_index(drop=True)

    ##### initilize TfidfVectorizer with NLTK's word_tokenize
    ##### function as tokenizer
    tfidf_vec=TfidfVectorizer(
        tokenizer=word_tokenize,
        token_pattern=None)
    #### fit count_vec on training data reviews
    tfidf_vec.fit(train_df.text)
    #transform training and validation data reviews
    x_train=tfidf_vec.transform(train_df.text)
    x_test=tfidf_vec.transform(test_df.text)
    # initialize logistic regression model
    model_logistic=linear_model.LogisticRegression(max_iter=1000)
    # fit the model on training data reviews and sentiment
    model_logistic.fit(x_train,train_df.target)
    preds_logtfidf=model_logistic.predict(x_test)
    ##Calculate accuracy
    if fold_!=4:
        accuracy=metrics.accuracy_score(test_df.target,preds_logtfidf)
        print(f"Fold:{fold_}")
        print(f"Accuracy={accuracy}")
        print("")
    